# ARPACK with PyLops

This notebook contains code written to showcase how to use ARPACK wrappers in scipy on PyLops linear operators to estimate eigenvalues. This code is now part of the scipy official documentation at http://scipy.github.io/devdocs/tutorial/arpack.html

In [5]:
import numpy as np
from scipy.linalg import eig, eigh
from scipy.sparse.linalg import eigs, eigsh

from pylops import MatrixMult, Diagonal, FirstDerivative
from pylops.signalprocessing import FFT

## Symmetric matrix

In [2]:
np.random.seed(0)
X = np.random.random((100,100)) - 0.5
X = np.dot(X, X.T) #create a symmetric matrix

In [3]:
evals_all, evecs_all = eigh(X)
evals_large, evecs_large = eigsh(X, 3, which='LM')

In [6]:
print(evals_all[-3:])
print(evals_large)
print(evals_all[-3:]-evals_large)

[29.1446102  30.05821805 31.19467646]
[29.1446102  30.05821805 31.19467646]
[-4.26325641e-14 -4.97379915e-14 -3.55271368e-14]


Let's now use MatrixMult pylops operators (symmetric so we can use eigsh)

In [7]:
N = 100
M = np.random.normal(0, 1, (N, N))
M = np.dot(M, M.T) #create a symmetric 
Mop = MatrixMult(M, dtype=np.float64)

evals_all, evecs_all = eigh(M)
evals_large, evecs_large = eigsh(Mop, 3, which='LM')

In [8]:
print(evals_all[-3:])
print(evals_large)
print(evals_all[-3:]-evals_large)

[328.15561917 345.39453702 360.63807832]
[328.15561917 345.39453702 360.63807832]
[ 9.09494702e-13 -2.27373675e-13  4.54747351e-13]


## Diagonal

Let's now use Diagonal pylops operators (still symmetric so we can use eigsh)

In [9]:
N = 10
d = (np.arange(N)+1.).astype(np.float64)
D = np.diag(d) 
Dop = Diagonal(d, dtype=np.float64)

evals_all, evecs_all = eigh(D)
evals_large, evecs_large = eigsh(Dop, 5, which='LA', maxiter=1e4)

In [10]:
print(evals_all[-5:])
print(evals_large)
print(evals_all[-5:]-evals_large)

[ 6.  7.  8.  9. 10.]
[ 6.  7.  8.  9. 10.]
[ 8.88178420e-16  0.00000000e+00 -1.77635684e-15 -1.77635684e-15
  0.00000000e+00]


In [11]:
evals_all, evecs_all = eigh(D)
evals_large, evecs_large = eigs(Dop, 5, which='LM', maxiter=1e4)

print(evals_all[-5:])
print(np.flipud(evals_large.real))
print(evals_all[-5:]-np.flipud(evals_large.real))

[ 6.  7.  8.  9. 10.]
[ 7.  6.  8.  9. 10.]
[-1.00000000e+00  1.00000000e+00  6.21724894e-15 -1.77635684e-15
  0.00000000e+00]


In [12]:
np.random.seed(0)

N = 100
d = np.random.normal(0, 1, N).astype(np.float64)
D = np.diag(d) 
Dop = Diagonal(d, dtype=np.float64)

evals_all, evecs_all = eigh(D)
evals_large, evecs_large = eigsh(Dop, 3, which='LA', maxiter=1e3)

In [13]:
print(evals_all[-3:])
print(evals_large)
print(evals_all[-3:]-evals_large)

print(np.dot(evecs_large.T, evecs_all[:,-3:]))

[1.9507754  2.2408932  2.26975462]
[1.9507754  2.2408932  2.26975462]
[ 6.66133815e-16  0.00000000e+00 -3.10862447e-15]
[[-1.00000000e+00 -1.08878742e-15  2.27332283e-16]
 [ 9.85498572e-16 -1.00000000e+00 -2.83515622e-14]
 [ 2.46314334e-16 -2.77019543e-14  1.00000000e+00]]


We can now create **an operator from scratch without Pylops for scipy doc**

In [14]:
import numpy as np
from scipy.sparse.linalg import LinearOperator

class Diagonal(LinearOperator):
    def __init__(self, diag, dtype='float32'):
        self.diag = diag
        self.shape = (len(self.diag), len(self.diag))
        self.dtype = np.dtype(dtype)

    def _matvec(self, x):
        return self.diag*x

    def _rmatvec(self, x):
        return self.diag*x
    
np.random.seed(0)
N = 100
d = np.random.normal(0, 1, N).astype(np.float64)
D = np.diag(d) 
Dop = Diagonal(d, dtype=np.float64)

evals_all, evecs_all = eigh(D)
evals_large, evecs_large = eigsh(Dop, 3, which='LA', maxiter=1e3)

print(evals_all[-3:])
print(evals_large)
print(evals_all[-3:]-evals_large)

print(np.dot(evecs_large.T, evecs_all[:,-3:]))

[1.9507754  2.2408932  2.26975462]
[1.9507754  2.2408932  2.26975462]
[6.66133815e-15 4.44089210e-15 4.44089210e-15]
[[-1.00000000e+00 -6.97479747e-16 -2.23452765e-16]
 [ 3.00600573e-16 -1.00000000e+00 -1.06719922e-14]
 [ 2.12381552e-16  1.05593224e-14 -1.00000000e+00]]


## Non-Symmetric matrix

Let's repeat the same exercise with a non-symettric MatrixMult

In [16]:
N = 100
M = np.random.normal(0, 1, (N, N)).astype(np.float64)
Mop = MatrixMult(M, dtype=np.float64)

evals_all, evecs_all = eig(M)
evals_large, evecs_large = eigs(Mop, 5, which='LR')

# as i find largest real need to extract largest real from evals_all
evals_all_real = evals_all.real
evals_all_real = np.sort(evals_all_real)

In [17]:
print(evals_all_real[-5:])
print(np.sort(evals_large.real))
print(evals_all_real[-5:]-np.sort(evals_large.real))

[7.68587676 7.72915784 7.72915784 9.37791307 9.37791307]
[7.68587676 7.72915784 7.72915784 9.37791307 9.37791307]
[-3.55271368e-15 -2.30926389e-14 -2.30926389e-14  3.01980663e-14
  3.01980663e-14]


## FirstDerivative

We now use a more complicated operator, the FirstDerivative which is not symmetric

In [18]:
N = 21
x = np.arange(N)

# dense
D = np.diag(0.5*np.ones(N-1),k=1) - np.diag(0.5*np.ones(N-1),-1)
D[0] = D[-1] = 0 # take away edge effects

# linear operator
Dop = FirstDerivative(N, dtype=np.float64)

# y = Dx
#y = np.dot(D,x)
#ylop = Dop*x
#print(y-ylop)

evals_all, evecs_all = eig(D)
evals_large, evecs_large = eigs(Dop, 4, which='LI')

# as i find largest real need to extract largest real from evals_all
evals_all_imag = evals_all.imag
isort_imag = np.argsort(np.abs(evals_all_imag))
evals_all_imag = evals_all_imag[isort_imag]
evals_large_imag = evals_large.imag
isort_imag = np.argsort(np.abs(evals_large_imag))
evals_large_imag = evals_large_imag[isort_imag]

In [19]:
print(evals_all_imag[-4:])
print(evals_large_imag)
print(evals_all_imag[-4:]-evals_large_imag)

[-0.95105652  0.95105652 -0.98768834  0.98768834]
[ 0.95105652 -0.95105652  0.98768834 -0.98768834]
[-1.90211303  1.90211303 -1.97537668  1.97537668]


In [20]:
class FirstDerivative(LinearOperator):
    def __init__(self, N, dtype='float32'):
        self.N = N
        self.shape = (self.N, self.N)
        self.dtype = np.dtype(dtype)
    def _matvec(self, x):
        y = np.zeros(self.N, self.dtype)
        y[1:-1] = (0.5*x[2:]-0.5*x[0:-2])
        return y
    def _rmatvec(self, x):
        y = np.zeros(self.N, self.dtype)
        y[0:-2] = y[0:-2] - (0.5*x[1:-1])
        y[2:] = y[2:] + (0.5*x[1:-1])
        return y

N = 21
D = np.diag(0.5*np.ones(N-1),k=1) - np.diag(0.5*np.ones(N-1),k=-1)
D[0] = D[-1] = 0 # take away edge effects
Dop = FirstDerivative(N, dtype=np.float64)

evals_all, evecs_all = eig(D)
evals_large, evecs_large = eigs(Dop, 4, which='LI')
evals_all_imag = evals_all.imag
isort_imag = np.argsort(np.abs(evals_all_imag))
evals_all_imag = evals_all_imag[isort_imag]
evals_large_imag = evals_large.imag
isort_imag = np.argsort(np.abs(evals_large_imag))
evals_large_imag = evals_large_imag[isort_imag]
evals_all_imag[-4:]
#array([-0.95105652  0.95105652 -0.98768834  0.98768834])
evals_large_imag
#array([0.95105652 -0.95105652  0.98768834 -0.98768834])

array([ 0.95105652, -0.95105652,  0.98768834, -0.98768834])

In [22]:
N = 21
D = np.diag(0.5*np.ones(N-1), k=1) - np.diag(0.5*np.ones(N-1), k=-1)
D[0] = D[-1] = 0 # take away edge effects
evals_all, evecs_all = eig(D)
evals_all_imag = evals_all.imag
isort_imag = np.argsort(np.abs(evals_all_imag))
evals_all_imag = evals_all_imag[isort_imag]
evals_all_imag[-4:]

array([-0.95105652,  0.95105652, -0.98768834,  0.98768834])